In [57]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("s076923/pytorch-transformer")

print("Path to dataset files:", path)

Path to dataset files: /home/user1/.cache/kagglehub/datasets/s076923/pytorch-transformer/versions/4


In [58]:
import os
import torch
from PIL import Image
from pycocotools.coco import COCO
from torch.utils.data import Dataset

In [59]:
class COCODataset(Dataset):
    def __init__(self, root, train, transform=None):
        super().__init__()
        directory = "train" if train else "val"
        annotations = os.path.join(root, "annotations", f"{directory}_annotations.json")
        self.coco = COCO(annotations)
        self.image_path = os.path.join(root, directory)
        self.transform = transform 
        self.categories = self.__get_categories()
        self.data = self.__load_data()
           
    def __get_categories(self):
        categories = {0 : 'background'}
        for category in self.coco.cats.values():
            categories[category['id']] = category['name']
        return categories
    
    def __load_data(self):
        data = []
        for _id in self.coco.imgs:
            file_name = self.coco.loadImgs(_id)[0]['file_name']
            image_path = os.path.join(self.image_path, file_name)
            image = Image.open(image_path).convert("RGB")

            boxes = []
            labels = []
            anns = self.coco.loadAnns(self.coco.getAnnIds(_id))

            for ann in anns:
                x, y, w, h = ann['bbox']
                boxes.append([x, y, x+w, y+h])
                labels.append(ann['category_id'])
            target = {
                'image_id' : torch.LongTensor([_id]),
                'boxes' : torch.FloatTensor(boxes),
                'labels' : torch.LongTensor(labels)
            }
            data.append([image, target])
        return data
       
    
    def __getitem__(self, index):
        image, target = self.data[index]
        if self.transform:
            image =self.transform(image)
        return image, target
        
    def __len__(self):
        return len(self.data)


In [60]:
from torchvision import transforms
from torch.utils.data import DataLoader


In [61]:
def collator(batch):
    return tuple(zip(*batch))

transform = transforms.Compose([
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(dtype=torch.float)
])

In [62]:
root = '/home/user1/.cache/kagglehub/datasets/s076923/pytorch-transformer/versions/4/datasets/coco'

In [63]:
train_dataset = COCODataset(root, train=True,transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, drop_last = True, 
          collate_fn = collator)

test_dataset = COCODataset(root, train=False,transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last = True, 
          collate_fn = collator)


loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [64]:
from torchvision import models 
from torchvision import ops 
from torchvision.models.detection import rpn , FasterRCNN

In [65]:
backbone = models.vgg16(weights="VGG16_Weights.IMAGENET1K_V1").features
backbone.out_channels = 512

In [66]:
anchor_generator = rpn.AnchorGenerator(
    sizes= ((32, 64, 128, 256, 512),),
    aspect_ratios = ((0.5, 1.0, 2.0),)
)

In [67]:
roi_pooler = ops.MultiScaleRoIAlign(
    featmap_names = ["0"],
    output_size=(7,7),
    sampling_ratio = 2
)

In [68]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [69]:
model = FasterRCNN(
    backbone=backbone, 
    num_classes = 3,
    rpn_anchor_generator = anchor_generator,
    box_roi_pool  = roi_pooler
).to(device)

In [70]:
from torch import optim

params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [ ]:
for epoch in range(5):
    cost = 0.0
    for idx, (images, targets)  in enumerate(train_dataloader):
        #image = image.to(device)
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        losses = sum([loss for loss in loss_dict.values()])

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        cost += losses 
    lr_scheduler.step()
    cost = cost / len(train_dataloader)
    print(f"Epoch : {epoch + 1:4d}, Cost : {cost:.3f}")

KeyboardInterrupt: 

: 